In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Memory management 
import gc 

#modelling
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from sklearn.calibration import calibration_curve, CalibratedClassifierCV

In [ ]:
# Load date sets.

train_data = pd.read_csv('/kaggle/input/tabular-playground-series-may-2022/train.csv')

test_data = pd.read_csv('/kaggle/input/tabular-playground-series-may-2022/test.csv')
test_ids = test_data['id'] 

# target


train_data.head()


In [ ]:
#check for unique values
for col in test_data:
    print(col)
    print(test_data[col].unique())

In [ ]:
#look for any NAN values and types

train_data.info()

In [ ]:
# check for dups Row

train_data[train_data.duplicated()]

In [ ]:
# column duplicates

train_data.apply(lambda x : x.duplicated().sum())

📝 Notes:

* f_27 has text with many Dups
* No nan Values. 

* f_29 and f_30 look like classes.

* f_29 [0 1]
* f_30 [0 1 2]

* f_27
text split or other features to extract.

In [ ]:
# Function to Clean up the data
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

#reload data

train_data = pd.read_csv('/kaggle/input/tabular-playground-series-may-2022/train.csv')

test_data = pd.read_csv('/kaggle/input/tabular-playground-series-may-2022/test.csv')



def clean(data):
    
    data = data.drop(['id'], axis = 1)
    
    
    # Split f_27 out by each of the 10 letters. 
    
    
    #This can be done better: 
    data['a1'] = data['f_27'].astype(str).str[0]
    data['a2'] = data['f_27'].astype(str).str[1]
    data['a3'] = data['f_27'].astype(str).str[2]
    data['a4'] = data['f_27'].astype(str).str[3]
    data['a5'] = data['f_27'].astype(str).str[4]
    data['a6'] = data['f_27'].astype(str).str[5]
    data['a7'] = data['f_27'].astype(str).str[6]
    data['a8'] = data['f_27'].astype(str).str[7]
    data['a9'] = data['f_27'].astype(str).str[8]
    data['a10'] = data['f_27'].astype(str).str[9]
 

    #map LabelEncoder for the new f_27 
    
    cols_to_map = ['a1','a2','a3','a4','a5','a6','a7','a8','a9','a10']
    for col in cols_to_map:
        data[col] = le.fit_transform(data[col])

        
    data = data.drop(['f_27'], axis = 1) # features broken out above. No longer needed. 

 
    
     
    
    
        
        
    return data
    
    
    
        
train_data = clean(train_data)
test_data = clean(test_data)

train_data.head()
    
       
y = train_data["target"]
X = train_data.drop("target", axis=1)

    
    
    
    
# X_train.head()
train_data.head()

In [ ]:
#train test split


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)





In [ ]:
EPOCHS = 5000

In [ ]:
model = lgb.LGBMClassifier(n_jobs = -1, n_estimators = EPOCHS)
model.fit(X_train,y_train, eval_set=[(X_test,y_test)], callbacks = [lgb.early_stopping(30)],eval_metric="auc")




In [ ]:
val_preds = model.predict_proba(X_test)
y_preds = model.predict_proba(X_train)

print("Intrinsic AUC:", roc_auc_score(y_train, y_preds[:,1]))
print("Validation AUC:", roc_auc_score(y_test, val_preds[:, 1] ))


In [ ]:
test_data.head()

In [ ]:
#Submission 

sub_preds = model.predict_proba(test_data)[:, 1]

sub_preds.size

In [ ]:
#check size of df must match

test_ids.size

In [ ]:
#m aking the submissions data frame

df = pd.DataFrame({'id':test_ids.values,
                 "target":sub_preds})

df.head()

In [ ]:
df.to_csv("MayTabPlaySub3.csv", index = False)